# Database import jupyter script

## I. Setup

### 1. Django Jupyter setup

In [1]:
import os, django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

### 2. Environment variables

In [2]:
from dotenv import load_dotenv
import os 
load_dotenv()

True

### 3. Other imports

In [3]:
# Import libraries
import psycopg
from tqdm import tqdm

# Import Django models
from account.models import *
from sales.models import *
from analysis.models import *

### 4. Connect to Postgres

In [4]:
# Connect to database
connection = psycopg.connect(
    dbname   = os.getenv("DATABASE_NAME"),
    user     = os.getenv("DATABASE_USER"),
    password = os.getenv("DATABASE_PASSWORD"),
    host     = os.getenv("DATABASE_HOST"),
    port     = os.getenv("DATABASE_PORT")
)

### 5. Function declaration

In [5]:
def printRecord(list):
    for record in list:
        print(record)

## II. Import từ Legacy vào Production database

### 1. Special Offers (done)

In [8]:
# Roll back to avoid error
# connection.rollback()

# Set up query
cur = connection.cursor()
cur.execute("""
SELECT * FROM "Company".specialoffer
ORDER BY specialofferid ASC
""")

# Result
rows = cur.fetchall()
# for row in rows:
#     print(row)
cur.close()

print("Done!")

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block

In [7]:
# Save records
for row in tqdm(rows):
    # Create object
    specialOffer = SpecialOffer(id=row[0],
                                Description=row[1],
                                Type=row[3],
                                StartDate=row[5],
                                EndDate=row[6],
                                MinQty=row[7],
                                MaxQty=row[8],
                                DiscountPct=row[2])
    
    # Save to database
    specialOffer.save()

  0%|                                                                                                                       | 0/16 [00:00<?, ?it/s]/home/dn/PycharmProjects/DA_HTTT_BE/.venv/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1665: RuntimeWarning: DateTimeField SpecialOffer.StartDate received a naive datetime (2011-05-01 00:00:00) while time zone support is active.
  warnings.warn(
/home/dn/PycharmProjects/DA_HTTT_BE/.venv/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1665: RuntimeWarning: DateTimeField SpecialOffer.EndDate received a naive datetime (2014-11-30 00:00:00) while time zone support is active.
  warnings.warn(
  6%|██████▉                                                                                                        | 1/16 [00:01<00:17,  1.19s/it]/home/dn/PycharmProjects/DA_HTTT_BE/.venv/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1665: RuntimeWarning: DateTimeField SpecialOffer.StartDate received a

In [8]:
printRecord(SpecialOffer.objects.all())

SpecialOffer object (1)
SpecialOffer object (2)
SpecialOffer object (3)
SpecialOffer object (4)
SpecialOffer object (5)
SpecialOffer object (6)
SpecialOffer object (7)
SpecialOffer object (8)
SpecialOffer object (9)
SpecialOffer object (10)
SpecialOffer object (11)
SpecialOffer object (12)
SpecialOffer object (13)
SpecialOffer object (14)
SpecialOffer object (15)
SpecialOffer object (16)


### 2. Products (done)

In [20]:
# Roll back to avoid error
connection.rollback()

# Set up query
cur = connection.cursor()
cur.execute("""
SELECT * FROM "Company".product
ORDER BY productid ASC
""")

# Result
rows = cur.fetchall()
# for row in rows:
#     print(row)
cur.close()

print("Done!")

Done!


In [21]:
# Save records
for row in tqdm(rows):
    product = Product(id=row[0],
                     Name=row[1],
                     Manufacturer=row[4],
                     Summary=row[3],
                     WarrantyPeriod=row[5],
                     RiderExperience=row[11],
                     Description=row[2],
                     Size="XXL",                # Size
                     Style=row[10],
                     StandardCost=row[12],           # TODO
                     ListPrice=row[13])              # TODO
                      
    product.save()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 294/294 [00:27<00:00, 10.67it/s]


In [9]:
printRecord(Product.objects.all())

Product object (709)
Product object (710)
Product object (711)
Product object (712)
Product object (713)
Product object (714)
Product object (715)
Product object (716)
Product object (717)
Product object (718)
Product object (719)
Product object (720)
Product object (721)
Product object (722)
Product object (723)
Product object (724)
Product object (725)
Product object (726)
Product object (727)
Product object (728)
Product object (729)
Product object (730)
Product object (732)
Product object (733)
Product object (734)
Product object (735)
Product object (736)
Product object (737)
Product object (738)
Product object (739)
Product object (740)
Product object (741)
Product object (706)
Product object (707)
Product object (708)
Product object (747)
Product object (751)
Product object (752)
Product object (753)
Product object (754)
Product object (755)
Product object (756)
Product object (757)
Product object (758)
Product object (759)
Product object (760)
Product object (743)
Product objec

### 3. Special offer - Product (done)

In [22]:
# Roll back to avoid error
connection.rollback()

# Set up query
cur = connection.cursor()
cur.execute("""
SELECT * FROM "Company".specialofferproduct
""")

# Result
rows = cur.fetchall()
# for row in rows:
#     print(row)
cur.close()

print("Done!")

Done!


In [23]:
# Save records
for row in tqdm(rows):
    try:
        # Get special offer object
        specialOffer = SpecialOffer.objects.get(id=row[0])

        # Get product object
        product = Product.objects.get(id=row[1])

        # Create a new special offer - product object
        specialOfferProduct = SpecialOfferProduct(SpecialOffer=specialOffer,
                                                  Product=product)

        specialOfferProduct.save()
    except Exception as e:
        pass

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 538/538 [02:31<00:00,  3.55it/s]


### 4. Territory

In [6]:
# Roll back to avoid error
connection.rollback()

# Set up query
cur = connection.cursor()
cur.execute("""
SELECT * FROM "Company".salesterritory
""")

# Result
rows = cur.fetchall()
# for row in rows:
#     print(row)
cur.close()

print("Done!")

Done!


In [7]:
# Save records
for row in tqdm(rows):
    territory = Territory(id=row[0],
                            Name = row[1],
                            Group = row[2],
                            SalesYTD = row[3],
                            SalesLastYear = row[4],
                 )
    ##territory.save()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 66894.80it/s]


### 5. Customer Store

In [12]:
# Roll back to avoid error
connection.rollback()

# Set up query
cur = connection.cursor()
cur.execute("""
SELECT * FROM "Company".store
ORDER BY "BusinessEntityID" ASC
""")

# Result
rows = cur.fetchall()
# for row in rows:
#     print(row)
cur.close()

print("Done!")

Done!


In [13]:
# Save records
for row in tqdm(rows):
    store = CustomerStore(id=row[0],
                    Name = row[1],
                    BusinessType = row[4],
                    Specialty = row[6],
                    AnnualSales = row[2],
                    AnnualRevenue = row[3],
                    YearOpened = row[5],
                    SquareFeet = row[7],
                    NumberOfEmployees = row[8],
                    City = row[11],
                    AddressLine1 = row[9],
                    AddressLine2 = row[10],
                    CountryRegionName = row[12],
                 )
    ##store.save()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 701/701 [00:00<00:00, 87233.56it/s]


In [9]:
printRecord(CustomerStore.objects.all())

Next-Door Bike Store: BM
Professional Sales and Service: BM
Riders Company: BM
The Bike Mechanics: BM
Nationwide Supply: BM
Area Bike Accessories: BM
Bicycle Accessories and Kits: BM
Clamps & Brackets Co.: BM
Valley Bicycle Specialists: OS
New Bikes Company: OS
Vinyl and Plastic Goods Corporation: OS
Top of the Line Bikes: OS
Fun Toys and Bikes: BM
Great Bikes : BM
Metropolitan Sales and Rental: BM
Irregulars Outlet: BM
Valley Toy Store: BM
Worthwhile Activity Store: BM
Purchase Mart: OS
Major Sport Suppliers: OS
Family's Favorite Bike Shop: BM
Global Plaza: BM
Imported and Domestic Cycles: BM
Systematic Sales: OS
eCommerce Bikes: OS
Mountain Toy Store: BM
Retail Sales and Service: BM
Designated Distributors: OS
Bold Bike Accessories: OS
Twin Cycles: BM
Tiny Bike Boutique: BM
Acclaimed Bicycle Company: OS
Serious Cycles: OS
Quality Bike Sales: OS
Novelty Bikes: OS
Distinctive Cycles Sales & Service: OS
Leading Bike Distributors: BS
Activity Center: BS
Brightwork Company: BS
Resale Serv

### 6. Customer Invidiual

In [6]:
# Roll back to avoid error
connection.rollback()

# Set up query
cur = connection.cursor()
cur.execute("""
SELECT * FROM "Company".individualcustomer
ORDER BY "BusinessEntityID" ASC
""")

# Result
rows = cur.fetchall()
# for row in rows:
#     print(row)
cur.close()

print("Done!")

Done!


In [7]:
# Save records
for row in tqdm(rows):
    individual = CustomerIndividual(id=row[0],
                        FirstName = row[2],
                        LastName = row[4],
                        MiddleName = row[3],
                        Title  = row[1],
                        EmailAddress = row[6],
                        PhoneNumber = row[5],
                        City = row[11],
                        AddressLine1 = row[9],
                        AddressLine2 = row[10],
                        CountryRegionName = row[12],
                 )
    ##individual.save()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18508/18508 [00:00<00:00, 164659.40it/s]


In [8]:
printRecord(CustomerIndividual.objects.all())

Carol Ann Rockne / carolann0@adventure-works.com
Scott Rodgers / scott10@adventure-works.com
Jim Rodman / jim4@adventure-works.com
Wyatt Anderson / wyatt10@adventure-works.com
Wyatt Thomas / wyatt11@adventure-works.com
Barry Raman / barry13@adventure-works.com
Barry Subram / barry14@adventure-works.com
Wyatt Jackson / wyatt12@adventure-works.com
Wyatt White / wyatt13@adventure-works.com
Wyatt Harris / wyatt14@adventure-works.com
Wyatt Martin / wyatt15@adventure-works.com
Wyatt Thompson / wyatt16@adventure-works.com
Barry Garcia / barry15@adventure-works.com
Wyatt Garcia / wyatt17@adventure-works.com
Wyatt Martinez / wyatt18@adventure-works.com
Barry Fernandez / barry16@adventure-works.com
Wyatt Robinson / wyatt19@adventure-works.com
Wyatt Clark / wyatt20@adventure-works.com
Wyatt Rodriguez / wyatt21@adventure-works.com
Wyatt Lewis / wyatt22@adventure-works.com
Barry Lopez / barry17@adventure-works.com
Wyatt Lee / wyatt23@adventure-works.com
Wyatt Walker / wyatt24@adventure-works.com
Da

### 7. Customer

In [8]:
# Roll back to avoid error
connection.rollback()

# Set up query
cur = connection.cursor()
cur.execute("""
SELECT * FROM "Company".customer
ORDER BY "CustomerID" ASC
""")

# Result
rows = cur.fetchall()
# for row in rows:
#     print(row)
cur.close()

print("Done!")

Done!


In [9]:
# Save records
for row in tqdm(rows):
    try:
        
        try:
            customerIndividual = CustomerIndividual.objects.get(id=row[1])
        except CustomerIndividual.DoesNotExist:
            customerIndividual = None

        # Attempt to get the store object
        try:
            store = CustomerStore.objects.get(id=row[2])
        except CustomerStore.DoesNotExist:
            store = None

        # Attempt to get the territory object
        try:
            territory = Territory.objects.get(id=row[3])
        except Territory.DoesNotExist:
            territory = None
        
        # Create a new special offer - product object
        customer = Customer(id = row[0],
                            Employee = None, 
                            CustomerIndividual = customerIndividual,
                            CustomerStore = store,
                            Territory = territory,
                            )

        customer.save()
    except Exception as e:
        print(e)
        pass

  0%|                                                                                                                                                    | 16/19820 [00:08<2:45:45,  1.99it/s]


KeyboardInterrupt: 

### 8. Employee

In [15]:
# Roll back to avoid error
connection.rollback()

# Set up query
cur = connection.cursor()
cur.execute("""
SELECT * FROM "Company".salesperson
ORDER BY "BusinessEntityID" ASC
""")

# Result
rows = cur.fetchall()
# for row in rows:
#     print(row)
cur.close()

print("Done!")

Done!


In [16]:
# Save records
for row in tqdm(rows):
    # Combine
    full_name = f"{row[1]} {row[2]} {row[3]}".strip()

    employee = Employee(
        id=row[0], 
        email=row[6],  
        name=full_name, 
        JobTitle=row[4], 
        PhoneNumber=row[5],  
        City=row[9],  # City
        AddressLine1=row[7], 
        AddressLine2=row[8],  
        CountryRegionName=row[10],  
        is_active=False,  
        is_staff=True,
        is_superuser=False,  
        isManager=False, 
        isEmployee=True,  
    )
    
    # Set the password (hashed) for the employee
    employee.set_password(row[5])  # Assuming row[5] contains the raw password
    
    # Save the employee instance to the database
    #employee.save()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:03<00:00,  5.21it/s]


### 9. Sales Order Header

In [19]:
# Roll back to avoid error
connection.rollback()

# Set up query
cur = connection.cursor()
cur.execute("""
SELECT * FROM "Company".salesorderheaderv1
ORDER BY "SalesOrderID" ASC
""")

# Result
rows = cur.fetchall()
# for row in rows:
#     print(row)
cur.close()

print("Done!")

Done!


In [21]:
# Save records
for row in tqdm(rows):
    try:
        try:
            employee = Employee.objects.get(id=row[5])
        except Employee.DoesNotExist:
            employee = None

        # Attempt to get the store object
        try:
            customer = Customer.objects.get(id=row[4])
        except Customer.DoesNotExist:
            customer = None

        # Attempt to get the territory object
        try:
            territory = Territory.objects.get(id=row[6])
        except Territory.DoesNotExist:
            territory = None
        
        # Create a new special offer - product object
        salesOrderHeader = SalesOrderHeader(id = row[0],
                            OrderDate=row[1],       
                            DueDate=row[2],         
                            ShipDate=row[3],        
                            ShipMethod=row[13],     
                            Comment=row[12],         
                            SubTotal=row[8],       
                            TaxAmt=row[9],          
                            Freight=row[10],         
                            TotalDue=row[11],
                            Employee = employee,
                            Customer = customer,
                            Territory = territory,
                            )

        salesOrderHeader.save()
    except Exception as e:
        print(e)
        pass

  3%|████▊                                                                                                                                             | 1033/31465 [00:00<00:08, 3411.20it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  6%|████████▋                                                                                                                                         | 1870/31465 [00:00<00:14, 2112.25it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
Notebo

 37%|█████████████████████████████████████████████████████▌                                                                                            | 11540/31465 [00:22<00:53, 375.78it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 38%|███████████████████████████████████████████████████████▋                                                                                          | 12003/31465 [00:24<00:50, 386.16it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
Notebo

 52%|███████████████████████████████████████████████████████████████████████████▎                                                                      | 16221/31465 [00:47<01:02, 243.58it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 52%|████████████████████████████████████████████████████████████████████████████▍                                                                     | 16469/31465 [00:48<00:54, 276.28it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
Notebo

 63%|███████████████████████████████████████████████████████████████████████████████████████████▌                                                      | 19737/31465 [01:11<00:55, 210.81it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 63%|████████████████████████████████████████████████████████████████████████████████████████████▌                                                     | 19954/31465 [01:12<00:49, 230.30it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
Notebo

 72%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                        | 22665/31465 [01:35<00:50, 175.63it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 73%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                        | 22834/31465 [01:36<00:47, 180.67it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
Notebo

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                            | 25281/31465 [01:58<00:40, 153.45it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 81%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                            | 25432/31465 [02:00<00:36, 166.50it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
Notebo

 86%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                   | 27208/31465 [02:20<00:38, 109.76it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 87%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                   | 27301/31465 [02:21<00:38, 107.70it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
Notebo

 91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉             | 28669/31465 [02:40<00:35, 78.19it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 92%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋            | 28804/31465 [02:41<00:16, 162.75it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
Notebo

 96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋     | 30325/31465 [03:01<00:07, 157.06it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 97%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎    | 30452/31465 [03:03<00:07, 139.11it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
Notebo

### 10. Sales Order Detail

In [6]:
# Roll back to avoid error
connection.rollback()

# Set up query
cur = connection.cursor()
cur.execute("""
SELECT * FROM "Company".salesorderdetail
ORDER BY "salesorderdetailid" ASC
""")

# Result
rows = cur.fetchall()
# for row in rows:
#     print(row)
cur.close()

print("Done!")

Done!


In [7]:
# Save records
for row in tqdm(rows):
    try:
        try:
            product = Product.objects.get(id=row[3])
        except Product.DoesNotExist:
            product = None

       
        try:
            specialOffer = SpecialOffer.objects.get(id=row[4])
        except SpecialOffer.DoesNotExist:
            specialOffer = None

        
        try:
            saleOrder = SalesOrderHeader.objects.get(id=row[0])
        except SalesOrderHeader.DoesNotExist:
            saleOrder = None
        
        
        salesOrderDetail = SalesOrderDetail(id = row[1],
                            SalesOrder = saleOrder,
                            Product = product,
                            SpecialOffer = specialOffer,
                            OrderQty = row[2],
                            UnitPrice = row[5],
                            UnitPriceDiscount = row[6],
                            LineTotal = row[7],
                            CarrierTrackingNumber = row[8],
                            )

        salesOrderDetail.save()
    except Exception as e:
        print(e)
        pass

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20188/20188 [2:23:40<00:00,  2.34it/s]


## II. Import từ "Production database" sang "Analysis database"

### 1. TerritoryDim

In [ ]:
# Query object (using Django query)
lst = Territory.objects.all()

# Save to analysis tables
for item in lst:
    try: 
        instance = TerritoryDim(id=item.id
                                Name=item.Name
                                Group=item.Group)
    except Exception as e:
        print(e)
        pass

### 2. EmployeeDim

### 3. CustomerDim

### 4. ProductDim

### 5. SpecialOfferDim

### 6. SalesOrderDetailFact


### 7. SalesOrderHeaderFact